In [1]:
import cv2
import numpy as np
import sys
import os
import math
#import matplotlib.pyplot as plt

#Shapes image moments, contours area matching
#approximate, convexity defect, convex hull, solidity factor, 
#image segmentation, Grab cut, graph cut GMMRF, watershed

In [9]:
# Extract all the contours from the image
def get_all_contours(img):
    ref_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(ref_gray, 127, 255, 0)

# Find all the contours in the thresholded image. The values
# for the second and third parameters are restricted to a
# certain number of possible values.

    im2, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, \
                                                cv2.CHAIN_APPROX_SIMPLE )
    return contours

# Extract reference contour from the image
def get_ref_contour(img):
    contours = get_all_contours(img)

# Extract the relevant contour based on area ratio. We use the
# area ratio because the main image boundary contour is
# extracted as well and we don't want that. This area ratio
# threshold will ensure that we only take the contour inside the image.

    for contour in contours:
        area = cv2.contourArea(contour)
        img_area = img.shape[0] * img.shape[1]
        if 0.1 < area/float(img_area) < 0.8:
            return contour


In [39]:
if __name__=='__main__':
# Boomerang reference image
    img1 = cv2.imread(sys.argv[1])
    
# Input image containing all the different shapes
    img2 = cv2.imread(sys.argv[2])
    
# Extract the reference contour
    ref_contour = get_ref_contour(img1)
    
# Extract all the contours from the input image
    input_contours = get_all_contours(img2)    
    
    contour_img = img2.copy()
    cv2.drawContours(contour_img, input_contours, -1, color=(0,0,0), \
                     thickness=3)
    cv2.imshow('Contours', contour_img)
    
    closest_contour = None
    min_dist = None
# Finding the closest contour
    for contour in input_contours:
# Matching the shapes and taking the closest one using
# Comparison method CV_CONTOURS_MATCH_I3 (second argument)
        ret = cv2.matchShapes(ref_contour, contour, 3, 0.0)
        print("Contour %d matchs in %f" % (i, ret))
        if min_dist is None or ret < min_dist:
            min_dist = ret
            closest_contour = contour
    
    cv2.drawContours(img2, [closest_contour], 0 , color=(0,0,0), thickness=3)
    cv2.imshow('Best Matching', img2)
    cv2.waitKey()

error: /build/opencv-L2vuMj/opencv-3.2.0+dfsg/modules/imgproc/src/color.cpp:9748: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [28]:
# Boomerang reference image
img1 = cv2.imread('./images/2.jpg')
    
# Input image containing all the different shapes
img2 = cv2.imread('./images/1.jpg')
    
# Extract the reference contour
ref_contour = get_ref_contour(img1)
    
# Extract all the contours from the input image
input_contours = get_all_contours(img2)    

ref_img = img1.copy()
contour_img = img2.copy()

cv2.drawContours(ref_img, ref_contour, -1, color=(0,0,0), \
                thickness=3)
cv2.drawContours(contour_img, input_contours, -1, color=(0,0,0), \
                thickness=3)

#cv2.imshow('Ref', ref_img)
#cv2.imshow('Contours', contour_img)

closest_contour = None
min_dist = None
i = 0
# Finding the closest contour
for contour in input_contours:
# Matching the shapes and taking the closest one using
# Comparison method CV_CONTOURS_MATCH_I3 (second argument)
    ret = cv2.matchShapes(ref_contour, contour, 3, 0.0)
    print("Contour %d matches in %f" % (i, ret))
    if ret > 0 and (min_dist is None or ret < min_dist):
        min_dist = ret
        closest_contour = contour
    i += 1
    
cv2.drawContours(img2, [closest_contour], 0 , color=(0,0,0), thickness=3)
cv2.imshow('Best Matching', img2)
cv2.waitKey()

Contour 0 matches in 0.000000
Contour 1 matches in 0.000000
Contour 2 matches in 0.722520
Contour 3 matches in 0.000000
Contour 4 matches in 0.000000
Contour 5 matches in 0.000000
Contour 6 matches in 0.000000
Contour 7 matches in 0.000000
Contour 8 matches in 0.000000
Contour 9 matches in 0.000000
Contour 10 matches in 0.000000
Contour 11 matches in 0.000000
Contour 12 matches in 0.000000
Contour 13 matches in 0.000000
Contour 14 matches in 0.000000
Contour 15 matches in 0.513358
Contour 16 matches in 0.000000
Contour 17 matches in 0.090587
Contour 18 matches in 0.000000
Contour 19 matches in 1.435032


255

In [29]:
# Input image containing all the different shapes
img1 = cv2.imread('./images/1.jpg')
# Extract all the contours from the input image
input_contours = get_all_contours(img1)
contour_img = img1.copy()

smoothen_contours = []
factor = 0.01

# Finding the closest contour
for contour in input_contours:
    epsilon = factor * cv2.arcLength(contour, True)
    smoothen_contours.append(cv2.approxPolyDP(contour, epsilon, True))

cv2.drawContours(contour_img, smoothen_contours, -1, color=(0,0,0), \
                 thickness=3)
cv2.imshow('Contours', contour_img)
cv2.waitKey()

255

In [37]:
img = cv2.imread('./images/pizza.jpg')

# Iterate over the extracted contours
# Using previous get_all_contours() method

for contour in get_all_contours(img):
# Extract convex hull from the contour
    factor = 0.01
    epsilon = factor * cv2.arcLength(contour, True)
    contour = cv2.approxPolyDP(contour, epsilon, True)
    hull = cv2.convexHull(contour, returnPoints=False)
# Extract convexity defects from the above hull
# Being a convexity defect the cavities in the hull segments
    defects = cv2.convexityDefects(contour, hull)
    if defects is None:
        continue
# Draw lines and circles to show the defects
    for i in range(defects.shape[0]):
        print(defects)
        start_defect, end_defect, far_defect, _ = defects[i,0]
        print(start_defect, end_defect, far_defect)
        start = tuple(contour[start_defect][0])
        end = tuple(contour[end_defect][0])
        far = tuple(contour[far_defect][0])
        print(start, end, far)
        cv2.circle(img, far, 5, [128,0,0], -1)
        cv2.drawContours(img, [contour], -1, (0,0,0), 3)

print(defects)
cv2.imshow('Convexity defects',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[  4   1   0 343]]]
4 1 0
(633, 419) (634, 417) (635, 418)
[[[    1     3     2 20330]]]
1 3 2
(595, 264) (652, 305) (573, 346)
[[[  1   3   2 256]]

 [[  4   0   5 256]]]
1 3 2
(355, 117) (355, 119) (354, 118)
[[[  1   3   2 256]]

 [[  4   0   5 256]]]
4 0 5
(354, 118) (354, 116) (355, 117)
[[[  2   0   3 256]]]
2 0 3
(504, 110) (506, 110) (505, 109)
None


In [59]:
# Input image containing all the shapes
img = cv2.imread('./images/figure.jpg')
img_orig = np.copy(img)
input_contours = get_all_contours(img)

solidity_values = []

# Compute solidity factors of all the contours
for contour in input_contours:
    area_contour = cv2.contourArea(contour)
    convex_hull = cv2.convexHull(contour)
    area_hull = cv2.contourArea(convex_hull)
    if area_hull == 0:
        continue
    else:
        solidity = float(area_contour)/area_hull
        solidity_values.append(solidity)

print(solidity_values)

# Clustering using KMeans
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS
solidity_values = np.array(solidity_values).reshape((len(solidity_values),1)).astype('float32')
compactness, labels, centers = cv2.kmeans(solidity_values, 2, None, criteria, 10, flags)
closest_class = np.argmin(centers)
print(centers, labels, closest_class)
print(solidity_values)
print(solidity_values[labels==closest_class])

output_contours = []

for i in solidity_values[labels==closest_class]:
    index = np.where(solidity_values==i)[0][0]
    print(index)
    output_contours.append(input_contours[index])

print(output_contours)
cv2.drawContours(img, output_contours, -1, (0,0,0), 3)
cv2.imshow('Output', img)

# Censoring

for contour in output_contours:
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img_orig, [box], 0, (0,0,0), -1)

cv2.imshow('Censored', img_orig)
cv2.waitKey()

[0.6869170552964691, 0.9795820321883257, 0.8075, 1.0, 0.6952919020715631, 0.8717728391204156, 0.5803085525987318, 0.9606570841889117, 0.8771888998778289, 0.8816450397655923, 0.8234677342484034]
[[ 0.65417254]
 [ 0.90022665]] [[0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]] 0
[[ 0.68691707]
 [ 0.97958201]
 [ 0.8075    ]
 [ 1.        ]
 [ 0.69529188]
 [ 0.87177283]
 [ 0.58030856]
 [ 0.96065706]
 [ 0.87718892]
 [ 0.88164502]
 [ 0.82346773]]
[ 0.68691707  0.69529188  0.58030856]
0
4
6
[array([[[378, 531]]], dtype=int32), array([[[597, 489]]], dtype=int32), array([[[499, 480]]], dtype=int32)]


255

In [55]:
# Draw rectangle based on the input selection
def draw_rectangle(event, x, y, flags, params):
    global x_init, y_init, drawing, top_left_pt, bottom_right_pt, img_orig
    # Detecting mouse button down event
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x_init, y_init = x, y
    
    # Detecting mouse movement
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            top_left_pt, bottom_right_pt = (x_init,y_init), (x,y)
            img[y_init:y, x_init:x] = 255 - img_orig[y_init:y, x_init:x]
            cv2.rectangle(img, top_left_pt, bottom_right_pt, (0,255,0), 2)

    # Detecting mouse button up event
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        top_left_pt, bottom_right_pt = (x_init,y_init), (x,y)
        img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
        cv2.rectangle(img, top_left_pt, bottom_right_pt, (0,255,0), 2)
        rect_final = (x_init, y_init, x-x_init, y-y_init)

        # Run Grabcut on the region of interest
        run_grabcut(img_orig, rect_final)

# Grabcut algorithm
def run_grabcut(img_orig, rect_final):
    # Initialize the mask
    mask = np.zeros(img_orig.shape[:2],np.uint8)

    # Extract the rectangle and set the region of
    # interest in the above mask
    x,y,w,h = rect_final
    mask[y:y+h, x:x+w] = 1

    # Initialize background and foreground models
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)

    # Run Grabcut algorithm
    cv2.grabCut(img_orig, mask, rect_final, bgdModel, fgdModel, 5,
    cv2.GC_INIT_WITH_RECT)

    # Extract new mask
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')

    # Apply the above mask to the image
    img_orig = img_orig*mask2[:,:,np.newaxis]

    # Display the image
    cv2.imshow('Output', img_orig)


drawing = False
top_left_pt, bottom_right_pt = (-1,-1), (-1,-1)

# Read the input image
img_orig = cv2.imread('./images/hand.jpg')
img = img_orig.copy()
cv2.namedWindow('Input')
cv2.setMouseCallback('Input', draw_rectangle)

while True:
    cv2.imshow('Input', img)
    c = cv2.waitKey(1)
    if c == 27:
        break

cv2.destroyAllWindows()

KeyboardInterrupt: 

In [50]:
#import cv2 as cv

img = cv.imread('coins.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)

img[markers == -1] = [255,0,0]

error: /build/opencv-L2vuMj/opencv-3.2.0+dfsg/modules/imgproc/src/color.cpp:9748: error: (-215) scn == 3 || scn == 4 in function cvtColor
